<a href="https://colab.research.google.com/github/inigmat/exupery/blob/main/CSP_binpacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install binpacking

In [2]:
# Check the csv data
import pandas as pd

CSV_URL = 'https://raw.githubusercontent.com/inigmat/exupery/main/files/bbs.csv'
data = pd.read_csv(CSV_URL)
data.head(5)

,index,Unique Mark,Dia,Length,Qty
0,0,100,10,2350,96
1,1,101,10,2800,24
2,2,102,10,3030,336
3,3,103,10,4230,144
4,4,200,12,450,320


In [3]:
from binpacking import to_constant_volume

# Constants
BAR_LENGTH = 11700
CSV_URL = CSV_URL


def calculate_required_bars(bar_schedule: pd.DataFrame, bar_length: int) -> tuple:
    """
    Calculates the required number of reinforcement bars and the cutting schemes.

    Args:
        bar_schedule (pd.DataFrame): DataFrame containing the length and quantity of bars.
        bar_length (int): Length of the bars.

    Returns:
        tuple: Required number of bars and the cutting schemes.
    """
    items = []
    # Extracting data from DataFrame
    for length, quantity in zip(bar_schedule['Length'], bar_schedule['Qty']):
        items.extend([length] * quantity)
    # Calculating the cutting schemes using bin packing algorithm
    cutting_scheme = to_constant_volume(items, bar_length)
    return cutting_scheme


def generate_cutting_table(cutting_data: dict) -> pd.DataFrame:
    """
    Generates a cutting table based on the cutting data.

    Args:
        cutting_data (dict): Dictionary containing cutting patterns for different diameters.

    Returns:
        cutting_scheme (pd.DataFrame): Cutting table with quantity, cuts, utilization, scrap, and diameter columns.
    """
    cutting_scheme = pd.DataFrame(columns=["Qty", "Cuts", "Utilization", "Scrap", "Dia"])
    # Processing cutting data for each diameter
    for dia, cutting_patterns in cutting_data.items():
        df_temp = pd.DataFrame(zip(cutting_patterns), columns=["Cuts"])
        # Calculate the quantity of each cutting scheme
        df_temp["Qty"] = df_temp["Cuts"].apply(lambda x: f"{cutting_patterns.count(x)}x")
        # Calculate the total length used by each cutting scheme
        df_temp["Utilization"] = df_temp["Cuts"].apply(sum)
        # Calculate the amount of scrap for each cutting scheme
        df_temp["Scrap"] = BAR_LENGTH - df_temp["Utilization"]
        # Add the diameter column
        df_temp["Dia"] = dia
        # Change the type of values in the Cuts column to drop duplicates
        df_temp["Cuts"] = df_temp["Cuts"].astype(str)
        # Concatenate df_temp with cutting_scheme
        cutting_scheme = pd.concat([cutting_scheme, df_temp], ignore_index=True)
    # Drop duplicate rows
    cutting_scheme = cutting_scheme.drop_duplicates()
    # Reset the index
    cutting_scheme = cutting_scheme.reset_index(drop=True)
    return cutting_scheme


def main() -> pd.DataFrame:
    # Read the data from CSV
    data = pd.read_csv(CSV_URL)
    cutting_data = {}
    # Process data for each unique diameter
    for dia in data['Dia'].unique():
        data_dia = data[data['Dia'] == dia]
        # Calculate the cutting schemes and store them in a dictionary
        cutting_patterns = calculate_required_bars(data_dia, BAR_LENGTH)
        cutting_data[dia] = cutting_patterns
        # Print results
        print(f'Required number of reinforcement bars with diameter {dia} mm: {len(cutting_patterns)} pcs.')
        percent = ((data_dia['Qty'] * data_dia['Length']).sum()/(len(cutting_patterns) * BAR_LENGTH)) * 100
        print(f'Total scrap: {round(100 - percent, 2)}%')
    # Generate the cutting table
    cutting_schedule = generate_cutting_table(cutting_data)
    return cutting_schedule


if __name__ == "__main__":
    # Execute the main function and store the resulting cutting table
    cutting_schedule = main()


Required number of reinforcement bars with diameter 10 mm: 168 pcs.
Total scrap: 2.32%
Required number of reinforcement bars with diameter 12 mm: 138 pcs.
Total scrap: 1.47%
Required number of reinforcement bars with diameter 16 mm: 143 pcs.
Total scrap: 3.76%
Required number of reinforcement bars with diameter 20 mm: 759 pcs.
Total scrap: 4.82%


# Showing the results

In [4]:
#Show the table
cutting_schedule

,Qty,Cuts,Utilization,Scrap,Dia
0,72x,"[4230, 4230, 3030]",11490,210,10
1,88x,"[3030, 3030, 3030, 2350]",11440,260,10
2,6x,"[2800, 2800, 2800, 2800]",11200,500,10
3,2x,"[2350, 2350, 2350, 2350]",9400,2300,10
4,32x,"[5280, 5280, 870]",11430,270,12
5,42x,"[3720, 3720, 3720, 450]",11610,90,12
6,1x,"[3720, 3720, 2300, 870, 870]",11480,220,12
7,15x,"[2300, 2300, 2300, 2300, 2300]",11500,200,12
8,1x,"[2300, 2300, 2300, 2300, 870, 870, 450]",11390,310,12
9,3x,"[870, 870, 870, 870, 870, 870, 870, 870, 870, ...",11310,390,12


# Downloading the results

In [5]:
from google.colab import files

# determining the name of the file
file_name = 'cutting_schedule.xlsx'

# saving the excel
cutting_schedule.to_excel(file_name)

# downloading the file
files.download('cutting_schedule.xlsx')
print('DataFrame is written to Excel File successfully. Save the file')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame is written to Excel File successfully. Save the file
